# DS - 3001 Final Project

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [5]:
covid = pd.read_csv("/Users/courtneynguyen2/Documents/myDS-3001/covid.csv")
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2537 entries, 0 to 2536
Data columns (total 22 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   fips_code                                   2537 non-null   int64  
 1   State                                       2537 non-null   object 
 2   County Name                                 2537 non-null   object 
 3   Staffed All Beds                            2537 non-null   float64
 4   Staffed ICU Beds                            2537 non-null   float64
 5   Licensed All Beds                           2537 non-null   float64
 6   All Bed Occupancy Rate                      2460 non-null   float64
 7   ICU Bed Occupancy Rate                      1410 non-null   float64
 8   Population                                  2537 non-null   float64
 9   Population (20+)                            2537 non-null   float64
 10  Population (

## **Question:** The CDC must determine the national healthcare system’s capacity to handle the COVID-19 pandemic and its severity. They would like to determine whether or not a county’s healthcare system capacity (the number of occupied beds) is at high occupancy (seen at 0.7 or more) based on county population, state, number of staffed beds per 1000 people, licensed all beds, etc.
